<a href="https://colab.research.google.com/github/JennyFrost/ML_and_LLMs/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai
!pip install nltk
!pip install -U spacy --no-cache-dir
!python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 138.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 143.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spa

In [ ]:
import openai
import re
import pandas as pd
from pandas.io.excel import ExcelWriter
import openpyxl
import datetime as dt
import string
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
import nltk
from nltk import sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import spacy
nlp = spacy.load("ru_core_news_lg", disable=['textcat'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# КОНСТАНТЫ

openai.api_key = ""

# НАЗВАНИЕ столбца С ТЕКСТОМ ДЛЯ GPT
message_col_name = 'Message'
preproc_col_name = 'Preprocessed message'
# ===========================================

# НАЗВАНИЕ СТОЛБЦА С ДАТОЙ ПИСЬМА
column_name_date = 'ReceivedTime'

# НАЗВАНИЕ СТОЛБЦА С РЕЗУЛЬТАТОМ GPT
neg_col_name = 'Is negative?'
type_neg_col_name = 'Type of negative'

# УКАЗЫВАЕМ ДАТУ, С КОТОРОЙ НАЧИНАТЬ СМОТРЕТЬ
start_date = '2023-09-06'
# ===========================================
start_date = dt.datetime.strptime(start_date, '%Y-%m-%d').date()

# СКОЛЬКО ЗАПИСЕЙ СМОТРЕТЬ В КАЖДОМ ФАЙЛЕ
size = 600
# ===========================================

# УКАЗЫВАЕМ ТЕКУЩИЙ МЕСЯЦ И ГОД
month = 'September'
year = '2023'

In [ ]:
files = !ls '/content/drive/My Drive/emails for sentiment'
files_list = [file_name.strip("\"' ") for f in files for file_name in re.split('\t|\s{2,5}', f) if file_name != '']
files_list

In [ ]:
for file_name in files_list:
    df = pd.read_excel('/content/drive/My Drive/emails for sentiment/' + file_name, parse_dates=[column_name_date])
    # df = pd.read_excel('/content/' + file_name, parse_dates=[column_name_date])
    df[column_name_date] = df[column_name_date].dt.date
    df = df[df[column_name_date] >= start_date]
    print(file_name, df.shape[0])

In [ ]:
pd.set_option('max_colwidth', 200)

In [ ]:
punct = '"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~⚫●⚫••■±°><☒□®©–ø™→–—−•■±°><☒□®©–ø™→'
cid_pattern = re.compile(r"\[cid:[0-9a-z\-]+(?:\.(?:jpg|png)@[0-9A-Z\.]+\])?")
digits_pattern = re.compile(r"[\d]+")
end_of_line_pattern = re.compile(r"\n+")
punct_pattern = re.compile(r'[{}]+'.format(punct))
file_path_pattern = re.compile(r'[A-Z]:?[\\/]+(?:[\w\-]+[\\/]+)+[\w\-]*')
url_pattern = re.compile(r'(?:https?://)?(?:www\.)?[a-zA-Z0-9@:%._\\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}(?:[a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$')
byte_pattern = re.compile(r"(?:\xa0|\u202f)")
english_text_pattern = re.compile(r"[a-zA-Z]+")
email_pattern=r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_pattern=r"(\+\d{1,2}\s)?\(?\d{1,4}\)?[\s.-]?\d{1,4}[\s.-]?\d{1,4}[\s.-]?\d{1,4}"
date_pattern=r"\b\d{1,2}[./-]\d{1,2}[./-]\d{2,4}\b"
whitespace_pattern = re.compile(r'\s{2,}')

In [ ]:
#pre-processing
def preprocess_text(text):
    sentences = sent_tokenize(text)
    docs = nlp.pipe(sentences)
    stopwords = nltk.corpus.stopwords.words('russian')
    lemmatized_text = [tok.text for doc in docs for tok in doc if tok.lemma_ not in stopwords
                                and tok.ent_type_ not in ('DATE', 'GPE', 'PERSON', 'MONEY')]
    if len(lemmatized_text) > 1250:
        lemmatized_text = lemmatized_text[:1250]
    text = re.sub(cid_pattern, "", ' '.join(lemmatized_text))
    text = re.sub(digits_pattern, "", text)
    text = re.sub(end_of_line_pattern, " ", text)
    text = re.sub(punct_pattern, " ", text)
    text = re.sub(byte_pattern, "", text)
    text = re.sub(url_pattern, "", text)
    text = re.sub(email_pattern, "", text)
    text = re.sub(phone_pattern, "", text)
    text = re.sub(english_text_pattern, "", text)
    text = re.sub(whitespace_pattern, ' ', text)
    return text.lower()

In [ ]:
system = """
You are a sales manager in a linguistic company. Your job is to analyze communication
with clients in the Russian and English languages.
If the message is negative, respond with the word "negative" and identify also the type of the claim. Respond with one of the following:
["poor quality translation", "missed deadlines", "troubles with payment", "system/technical failures", "translation or layout quality", "lack of resources", "shifting deadlines", "notarization issues", "urgency", "delivery issues", "poor service"].
The output for negative messages should be as follows:
"negative", "poor quality translation"
If the message if not negative, respond with "ok".
"""

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    resp = openai.ChatCompletion.create(**kwargs).choices[0].message.content
    return resp

In [ ]:
def apply_gpt(message, prompt=system, model='gpt-4'):
    resp = completion_with_backoff(
                model=model,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": message},
                ],
                temperature=0,
                max_tokens=30
                )
    if resp not in ('ok', '"ok"'):
        print('message: ', message)
        print('GPT response: ', resp)
    return resp

In [ ]:
def process_dataset(df, model: str, prompt: str, in_file: str, month: str = month, year: str = year):
  df[preproc_col_name] = df[message_col_name].astype('str').apply(preprocess_text)
#   col_name_gpt = 'ChatGPT_new' if model=="gpt-4-0314" else 'ChatGPT'
#   prompt = system2 if model=="gpt-4-0314" else system
#   is_basic = 'new' if model=="gpt-4-0314" else 'basic'
  df = df[df[preproc_col_name].str.strip() != '']
  df[neg_col_name] = df[preproc_col_name].apply(apply_gpt)
  print(f'{len(df)} records processed by {model}, file {file_name}', end='\n')
  df = df[df[neg_col_name].str.contains('negative')]
  df[type_neg_col_name] = df[neg_col_name].apply(lambda x: x.split(',')[1].strip('" ') if len(x.split(',')) > 1 else '')
  df[neg_col_name] = df[neg_col_name].apply(lambda x: x.split(',')[0].strip('" '))
  df.drop(preproc_col_name, axis=1, inplace=True)
#   file_name = f'/content/drive/My Drive/results sentiment/{in_file[:-5]}_sentiment_{month}_{year}_{is_basic}.xlsx'
#   file_name = f'/content/drive/My Drive/results sentiment/{in_file[:-5]}_sentiment_{month}_{year}.xlsx'
#   try:
#     with pd.ExcelWriter(file_name, mode='a', engine='openpyxl') as writer:
#       df.to_excel(writer, index=False)
#   except FileNotFoundError:
#     with pd.ExcelWriter(file_name, mode='w', engine='openpyxl') as writer:
#       df.to_excel(writer, index=False)
  print(f'Количество негативных писем: ', df.shape[0], end='\n')
  return df

In [ ]:
for file_name in files_list:
    df = pd.read_excel('/content/drive/My Drive/emails for sentiment/' + file_name, parse_dates=[column_name_date])
    print()
    # df = pd.read_excel('/content/' + file_name, parse_dates=[column_name_date])
    df[column_name_date] = df[column_name_date].dt.date
    df = df[df[column_name_date] >= start_date]
    # df = process_dataset(df, model="gpt-3.5-turbo", prompt=system, in_file=file_name)
    # df_negative = df[df['ChatGPT'].str.contains('negative', case=False)]
    # df_final = process_dataset(df_negative, model="gpt-4-0314", prompt=system2, in_file=file_name)
    df_final = process_dataset(df, model="gpt-4", prompt=system, in_file=file_name)
    df_final.to_excel(f'/content/drive/My Drive/results sentiment/{file_name[:-5]}_negative_{month}_{year}.xlsx', index=False)
    print(f'Обработан файл {file_name}', end='\n\n')
    print('==================================================================================', end='\n\n')

In [ ]:
df_neg = pd.DataFrame()
files = !ls '/content/drive/My Drive/results sentiment'
for file_name in files:
    if 'negative' in file_name:
        file_name = file_name.strip("'")
        df = pd.read_excel(f'/content/drive/My Drive/results sentiment/{file_name}')
        df_neg = pd.concat([df_neg, df])

In [ ]:
df_neg.shape

(711, 7)

In [ ]:
df_neg.to_excel(f'/content/drive/My Drive/results sentiment/all_negative_{month}_{year}.xlsx', index=False)

In [ ]:
df_neg = df_final[df_final['GPT'].str.contains('negative')]
df_neg.shape

(109, 8)

In [ ]:
df_neg.head()

,From,To,ReceivedTime,Subject,Message,Entire correspondence,Conversation ID,GPT
8299,m.eganian@awatera.com,evgeny.tonkikh@novartis.com,2023-08-10,RE: перевод,евгений к сожалению оба перевода будут уже завтра г москва бц золото ул золотая,"Евгений, к сожалению, оба перевода будут уже завтра \n\n\n\n\n\n\nMargarita Eganian\n\nManager of Support Team \n\n \n\nг. Москва, БЦ ""Золото"", \nул. Золотая, 11\n\n\nAWATERA.COM [http://awatera.c...",AAQkAGNjMDhjYjQ0LTE4NWYtNDkyZC04OGE4LTI1ZDc3Mzc5M2U4YwAQABfvUracKT5LkIBcWx0sqBM=,"""slightly negative"", ""missed deadlines"""
8313,m.eganian@awatera.com,Marina.Poroikova@sanofi.com,2023-08-10,S_SNF-3794,марина добрый вечер ! направляю готовый перевод прошу прощения за задержку спасибо ! г москва бц золото ул золотая,"Марина, добрый вечер!\nНаправляю готовый перевод.\n\nПрошу прощения за задержку.\n\nСпасибо!\n\n\n\n\n\n\n\n\nMargarita Eganian\n\nManager of Support Team \n\n \n\nг. Москва, БЦ ""Золото"", \nул. Зо...",AAQkAGNjMDhjYjQ0LTE4NWYtNDkyZC04OGE4LTI1ZDc3Mzc5M2U4YwAQALa_rqHT3khGquQdIuQ5xSI=,"""slightly negative"", ""missed deadlines"""
8336,Ekaterina.Fatkina@sanofi.com,m.eganian@awatera.com,2023-08-11,FW: Смета по заказу S_SNF-4103,маргарита добрый день ! подскажите пожалуйста по оплате заказа в личном кабинете он указан в статусе “ ” и я не могу соответственно согласовать заказ для оплаты в нашей системе не могли бы вы поме...,"Маргарита, добрый день!\n\n \n\nПодскажите, пожалуйста, по оплате заказа S_SNF-4103: в личном кабинете он указан\nв статусе “In Progress”, и я не могу, соответственно согласовать заказ для\nоплаты...",AAQkAGNjMDhjYjQ0LTE4NWYtNDkyZC04OGE4LTI1ZDc3Mzc5M2U4YwAQACTsa_QgKLVBijNolsrgU58=,"""slightly negative"", ""troubles with payment"
8366,m.eganian@awatera.com,Ekaterina.Fatkina@sanofi.com,2023-08-11,RE: Смета по заказу S_SNF-4103,екатерина немного задержали направляю перевод спасибо ! г москва бц золото ул золотая,"Екатерина, немного задержали, направляю перевод.\n\n\nСпасибо!\n\n\n\n\n\n\nMargarita Eganian\n\nManager of Support Team \n\n \n\nг. Москва, БЦ ""Золото"", \nул. Золотая, 11\n\n\nAWATERA.COM [http:/...",AAQkAGNjMDhjYjQ0LTE4NWYtNDkyZC04OGE4LTI1ZDc3Mzc5M2U4YwAQACTsa_QgKLVBijNolsrgU58=,"""slightly negative"", ""missed deadlines"""
8380,Polina.Kholina@sanofi.com,m.eganian@awatera.com,2023-08-11,RE: Смета по заказу S_SNF-4105,маргарита добрый вечер ! мб это баг заказ пропал из личного кабинета не могу завести ро спасибо заранее !,"Маргарита, добрый вечер!\n\n \n\nМб это баг, заказ пропал из личного кабинета, не могу завести РО.\n\nСпасибо заранее!\n\n \n\nBest regards,\n\nPolina Alekseeva (Kholina)\n\nRegulatory Affairs Ass...",AAQkAGNjMDhjYjQ0LTE4NWYtNDkyZC04OGE4LTI1ZDc3Mzc5M2U4YwAQAP93lvQPYWJKnBVCh05zl8s=,"The sentiment of the message is ""slightly negative"
